### CMOS LED test 
check synchronization to TimePixCam

In [ ]:
from beamtimedaqaccess import BeamtimeDaqAccess 
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
root_dir = '/asap3/flash/gpfs/bl1/2019/data/11006902/raw/hdf/online-0/'
image_dset_name = '/FL1/Experiment/BL1/CAMP/VMI CMOS camera 1/image'

run_number = 32267

In [ ]:
daq = BeamtimeDaqAccess.create(root_dir)

In [ ]:
def section_ID_intervals(total_interval):
    assert isinstance(total_interval, tuple)
    assert len(total_interval) == 2
    assert total_interval[0] <= total_interval[1]
    section_length = 1000  
    sections = [(start, start + section_length) for start in range(total_interval[0], total_interval[1], section_length)]
    sections[-1] = (sections[-1][0], total_interval[1])  # fix last section
    return sections

In [ ]:
total_trainID_inteval = daq.pulseIdIntervalOfRun(image_dset_name, run_number)
sections = section_ID_intervals(total_trainID_inteval)

image_integral = []
for i in range(len(sections)):
    print('working on section:',i+1, '/', len(sections))
    images = daq.valuesOfInterval(image_dset_name, sections[i])
    for j in range(images.shape[0]):
        image_integral.append(np.sum(images[j]))


In [ ]:
counts = plt.hist(image_integral, 1000)
plt.axvline(1.25E6, c='r')
# plt.xlim(1E6,1.5E6)
plt.show()

In [ ]:
trainIDs = np.linspace(total_trainID_inteval[0], total_trainID_inteval[1],total_trainID_inteval[1]-total_trainID_inteval[0])

print(len(trainIDs), len(image_integral))

In [ ]:
light = []
for i in range(len(image_integral)):
    if image_integral[i] > 1.25E6:
        light.append(1)
    else:
        light.append(0)   

In [ ]:
fig = plt.figure(figsize = (10,5))
plt.plot(trainIDs, light,'o')
plt.xlabel('trainID')
plt.ylabel('light [on/off]')
plt.xlim(5130+6.0964E8,5160+6.0964E8)
plt.show()

In [ ]:
#np.savez('cmos_light_'+str(run_number),trainIDs = trainIDs, light = light)